# Direct and Indirect Modes
KERI provides a secure way to manage Identifiers and track control using verifiable logs of key events (KEL). How these logs are shared and verified between the controller and someone verifying that identifier leads to two operational modes: Direct and Indirect.

## Direct Mode

Direct Mode is a one-to-one approach, similar to a direct conversation.

The controller shares their Key Event Log directly with a specific validator, who verifies the signatures and checks the cryptographic chain to ensure integrity. The method of sharing the KEL is implementation-dependent.

Trust is based solely on verifying the controller’s own signatures, as the validator directly receives and validates the KEL.

This mode is an option for interactions where both parties can connect directly, even if only occasionally, and need to be online to exchange new events or updates.  

## Indirect Mode

Indirect Mode is the highly-available approach, similar to using a trusted public bulletin board instead of direct messaging.

It’s for scenarios where the controller may be offline or needs to support many validators at once. Rather than relying on direct communication, it introduces infrastructure to keep the event log reliably accessible.

Trust extends beyond the controller’s signature, relying on a network of Witnesses that verify, sign, and store events.

This mode is ideal for public identifiers, always-verifiable services, one-to-many interactions, or any situation where the controller can’t be constantly online.



## Out-of-Band Introductions (OOBIs)

When operating in indirect mode you need a way to tell others *where* they can find information about it, like its Key Event Log (KEL). This is where Out-of-Band Introductions (OOBIs) come in.

**What is an OOBI?**

An OOBI is a **discovery mechanism** used in KERI. Its primary purpose is to link a specific KERI AID  to a network location (a URL or URI) where information about that identifier can potentially be found.

**Format:**

The simplest form of an OOBI pairs an AID with a URL. It can be represented in various ways, For example:

`("http://8.8.5.6:8080/oobi", "EaU6JR2nmwyZ-i0d8JZAoTNZH3ULvYAfSVPzhzS6b5CM")`

This OOBI suggests that information related to the AID `EaU6JR2nmwyZ-i0d8JZAoTNZH3ULvYAfSVPzhzS6b5CM` might be available at the service endpoint `http://8.8.5.6:8080/oobi`.

**Crucial Point: OOBIs are NOT Trusted**

This is the most important concept to understand about OOBIs:

* **The OOBI itself is untrusted.** Just because you receive an OOBI doesn't mean the link between the URL and the AID is legitimate.
* **The data at the OOBI endpoint is untrusted.** Any information retrieved from the URL provided in the OOBI *must* be independently verified using KERI's cryptographic mechanisms (like checking signatures against the AID's KEL).

**Discovery vs. Trust**

OOBIs cleverly separate the problem of *discovery* from the problem of *trust*:

1.  **Discovery (Out-of-Band):** OOBIs leverage existing internet infrastructure (like web servers, DNS, search engines, QR codes, emails) to share these `(url, aid)` pairs. This is considered "out-of-band" because it uses mechanisms outside the core KERI protocol guarantees.
2.  **Trust (In-Band):** Once you use the OOBI's URL to retrieve data (e.g., a KEL), you *must* use KERI's standard "in-band" cryptographic verification processes to establish trust in that data. You verify the signatures and the event history according to KERI rules.

In short: **Discovery via URI, Trust via KERI.**

OOBIs provide a flexible way to announce where KERI information might be found, relying on KERI's robust verification methods to ensure the trustworthiness of the information once retrieved.

Add a note on said.


# Witnesses

Witnesses are entities designated by the controller within their key event log, acting much like trusted notaries. Their role is to receive key events directly from the controller, verify the controller’s signature, and check that each event aligns with the historical log they maintain.

Once a witness confirms an event is valid and encounters it for the first time in sequence, it generates a receipt by signing the event. The witness then stores both the original event and its own receipt—alongside receipts from other witnesses—in a local copy of the log known as the **Key Event Receipt Log (KERL)**.

Witnesses play a critical role in ensuring the system’s reliability and integrity. They provide availability by forming a distributed service that validators can query to access the controller’s log, even if the controller itself is unavailable. Additionally, they help ensure consistency: since honest witnesses only sign the first valid version of an event they observe, it becomes significantly harder for a controller to present conflicting log versions. By gathering enough receipts from these witnesses, validators gain strong assurance that the event history they’re working with is the one broadly agreed upon.

<div class="alert alert-prymary">
  <b>📝 SUMMARY</b><hr>
KERI supports two main modes for sharing and verifying identifiers: Direct and Indirect.

**Direct Mode** is a peer-to-peer approach where the controller shares their Key Event Log (KEL) directly with a validator. Trust is based solely on the controller’s own cryptographic signatures. It works best for one-to-one interactions where both parties are online.

**Indirect Mode** is designed for scalability and availability. It introduces a trusted infrastructure of **Witnesses** — designated entities that receive, verify, and sign events from the controller. These witnesses maintain a copy of the event log (KERL), making it available even when the controller is offline. Validators trust not only the controller’s signatures but also the receipts from multiple witnesses, ensuring log consistency and resistance to tampering.
</div>

WIP WIP

In [ ]:
# Imports and Utility functions
import subprocess

def clear_keri():
    path = "/usr/local/var/keri/"
    confirm = input("🚨 This will clear your keystore. Are you sure? (y/n): ")
    if confirm.lower() == "y":
        print("Proceeding...")
        try:
            subprocess.run(["rm", "-rf", path], check=True)
            print(f"✅ Successfully removed: {path}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Error removing {path}: {e}")
    else:
        print("Operation cancelled.")

we have a demo witness network deployed already
explain the demo witness

test the witness with curl, KEL of the witness AID

explain oobis


In [ ]:
!curl -s http://witness-demo:5642/oobi/BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha

Create Key Store
explain the config file
explain loading oobis
explain keri/cf/ (note)

In [ ]:
keystore_name="tests-key-store"

keystore_passcode="xSLg286d4iWiRg2mzGYca"

salt="0ABeuT2dErMrqFE5Dmrnc2Bq"

!kli init --name {keystore_name} --passcode {keystore_passcode} --salt {salt} \
    --config-dir ./config \
    --config-file keystore_init_config.json


In [ ]:
!kli contacts list --name {keystore_name} --passcode {keystore_passcode}

Incept transferable AID
explain we are not using config params, explain the config.json
explain witness receipts

In [ ]:
aid_alias_transferable = "aid-transferable"

!kli incept --name {keystore_name} --alias {aid_alias_transferable} --passcode {keystore_passcode} \
    --file ./config/aid_inception_config.json

In [ ]:
!kli status --name {keystore_name} --alias {aid_alias_transferable} --passcode {keystore_passcode} --verbose

In [ ]:
!curl -s http://witness-demo:5642/oobi/ED5HLfY2h7pVa_MgLsF1CkK4CHWro4tOTOgvndmiuztK

In [ ]:
aid_alias_non_transferable = "aid-non-transferable"

!kli incept --name {keystore_name} --alias {aid_alias_non_transferable} --passcode {keystore_passcode} \
    --icount 1 --isith 1 --ncount 1 --nsith 1 --toad 0

In [ ]:
!kli query --name {keystore_name} --passcode {keystore_passcode} --alias {aid_alias_transferable} --prefix "EJq-DYl9EQVlY1lShETUWLQuDEcVdRkWXfkkGBNDugjZ"

In [ ]:
!curl -s http://witness-demo:5642/oobi/BEG5uWt6xB94bIkdGUCjYcBf_ryDgPa7t1GUtVc7lerw/witness/BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha

In [ ]:
!kli status --name {keystore_name} --passcode {keystore_passcode} --alias {aid_alias_transferable} --verbose

In [ ]:
!kli query --name {keystore_name} --passcode {keystore_passcode} --alias {aid_alias_transferable} --prefix "BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha"

In [ ]:
!kli query --name {keystore_name} --passcode {keystore_passcode} --alias {aid_alias_non_transferable} --prefix "BEG5uWt6xB94bIkdGUCjYcBf_ryDgPa7t1GUtVc7lerw"

In [ ]:
!kli status --name {keystore_name} --passcode {keystore_passcode} --alias {aid_alias_non_transferable} --verbose